# Module 3 - Interactive Dashboard with Ploty Dash

In [1]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

<ipython-input-1-89e3ae8eb04c>:4: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
<ipython-input-1-89e3ae8eb04c>:5: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


In [2]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

In [3]:
# Create a dash application
app = dash.Dash(__name__)

In [4]:
# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown',
                                            options=[{'label': 'All Sites', 'value': 'ALL'},
                                                    {'label': 'CCAFS LC-40 - Cape Canaveral Launch Complex 40', 'value': 'CCAFS LC-40'},
                                                    {'label': 'VAFB SLC-4E - Vandenberg Space Launch Complex 4E', 'value': 'VAFB SLC-4E'},
                                                    {'label': 'KSC LC-39A - Kennedy Space Center Launch Complex 39A', 'value': 'KSC LC-39A'},
                                                    {'label': 'CCAFS SLC-40 - Cape Canaveral Space Launch Complex 40', 'value': 'CCAFS SLC-40'}
                                                    ],
                                            value='ALL',
                                            placeholder='Select a Launch Site here',
                                            searchable=True
                                            ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(
                                    id='payload-slider',
                                    min=0,
                                    max=10000,
                                    step=1000,
                                    marks={0: '0 kg',
                                        1000: '1000 kg',
                                        2000: '2000 kg',
                                        3000: '3000 kg',
                                        4000: '4000 kg',
                                        5000: '5000 kg',
                                        6000: '6000 kg',
                                        7000: '7000 kg',
                                        8000: '8000 kg',
                                        9000: '9000 kg',
                                        10000: '10000 kg'
                                        },
                                    value=[min_payload, max_payload]
                                    ),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

In [5]:
# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(
            Output(component_id='success-pie-chart', component_property='figure'),
            Input(component_id='site-dropdown', component_property='value')
            )
def get_pie_chart(entered_site):
    if entered_site == 'ALL':
        fig = px.pie(
            data_frame=spacex_df,
            values='class',
            names='Launch Site',
            title='Successful Launches by Site',
            )
    else:
        filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site].groupby('class')['Flight Number'].count().reset_index()
        filtered_df['Booster Landing'] = ['Successful Landing' if cls == 1 else 'Unsuccessful Landing' for cls in filtered_df['class'].to_list()]
        filtered_df.rename(columns={'Flight Number': 'Total Number of Flights'}, inplace=True)
        fig = px.pie(
            data_frame=filtered_df,
            values='Total Number of Flights',
            names='Booster Landing',
            title='Success Rate for Launch Site ' + entered_site,
            color='Booster Landing',
            color_discrete_map={'Successful Landing': 'green', 'Unsuccessful Landing': 'red'}
            )
    return fig

In [6]:
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(
            Output(component_id='success-payload-scatter-chart', component_property='figure'),
            Input(component_id='site-dropdown', component_property='value'),
            Input(component_id='payload-slider', component_property='value')
            )
def get_scatter_chart(entered_site, range_kg):
    if entered_site == 'ALL':
        filtered_df = spacex_df[
            (spacex_df['Payload Mass (kg)'] >= range_kg[0])
            & (spacex_df['Payload Mass (kg)'] <= range_kg[1])
            ]
        title = 'Correlation between Payload and Landing Success for all Sites'
    else:
        filtered_df = spacex_df[
            (spacex_df['Launch Site'] == entered_site)
            & (spacex_df['Payload Mass (kg)'] >= range_kg[0])
            & (spacex_df['Payload Mass (kg)'] <= range_kg[1])
            ]
        title = 'Correlation between Payload and Landing Success for Launch Site ' + entered_site
    fig = px.scatter(
        data_frame=filtered_df,
        x='Payload Mass (kg)',
        y='class',
        color='Booster Version Category',
        title=title
        )
    return fig

In [7]:
# Run the app
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [13/Dec/2021 12:18:28] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Dec/2021 12:18:28] "GET /_dash-component-suites/dash/deps/polyfill@7.v2_0_0m1639393420.12.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [13/Dec/2021 12:18:28] "GET /_dash-component-suites/dash/deps/react-dom@16.v2_0_0m1639393420.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [13/Dec/2021 12:18:28] "GET /_dash-component-suites/dash/deps/react@16.v2_0_0m1639393420.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [13/Dec/2021 12:18:28] "GET /_dash-component-suites/dash/dash-renderer/build/dash_renderer.v2_0_0m1639393419.min.js HTTP/1.1" 200 -
127.0.0.1 - - [13/Dec/2021 12:18:28] "GET /_dash-component-suites/dash/dcc/dash_core_components-shared.v2_0_0m1639393420.js HTTP/1.1" 200 -
127.0.0.1 - - [13/Dec/2021 12:18:28] "GET /_dash-component-suites/dash/deps/prop-types@15.v2_0_0m1639393420.7.2.min.js HTTP/1.1" 200 -
127.0.0.1 - - [13/Dec/2021 12:18:28] "GET /_dash-component-s